In [1]:
import pandas as pd
import numpy as np

In [10]:
movies = pd.read_csv('ml-1m/movies.dat', 
                     delimiter='::', 
                     names=['MovieID','Title','Genres'],
                     header=None,
                     engine='python')
users = pd.read_csv('ml-1m/users.dat', 
                     delimiter='::', 
                     names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
                     header=None,
                     engine='python')
ratings = pd.read_csv('ml-1m/ratings.dat', 
                     delimiter='::', 
                     names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
                     header=None,
                     engine='python')

In [11]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
# 1
# movies["Title"].str[-5:-1]
# 2
# movies["Title"].str.extract(r'\((\d{4})\)')

In [16]:
movies["Year"] = movies["Title"].str[-5:-1].astype(int)
movies.head()

,MovieID,Title,Genres,Year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


### A. ile jest wszystkich filmów

In [20]:
result = movies.shape[0]
result

3883

In [21]:
len(movies)

3883

In [29]:
np.count_nonzero(pd.unique(movies["Title"]))

3883

### B. ile filmów powstało w poszczególnych latach

In [35]:
result = movies.groupby("Year").size().reset_index().rename({0: "count"}, axis=1)
result.head()

,Year,count
0,1919,3
1,1920,2
2,1921,1
3,1922,2
4,1923,3


### C. jak wygląda rozkład płci oraz grup wiekowych wśród użytkowników

In [37]:
pd.crosstab(users["Gender"],users["Age"])

Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,78,298,558,338,189,146,102
M,144,805,1538,855,361,350,278


### D. jaki gatunek filmowy jest najczęstszy

In [38]:
movies.head()

,MovieID,Title,Genres,Year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [71]:
result = movies["Genres"].str.split("|", expand=True).stack().reset_index().rename({0: "Genre"}, axis=1)
result.groupby("Genre").size().sort_values(ascending=False).reset_index().rename({0: "count"}, axis=1).head(1)

,Genre,count
0,Drama,1603


### E. jaki jest najlepszy film wszechczasów, (najlepszy, czyli ma najwyższą średnią ocenę) - to zadanie możesz rozwiązać wykonując złączenie (join) zbioru movies i ratings

In [133]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [138]:
average_rating = pd.pivot_table(ratings[["MovieID", "Rating"]], index=["MovieID"], aggfunc="mean").reset_index()
max_rating = float(average_rating.sort_values(by="Rating", ascending=False).reset_index(drop=True).loc[0,["Rating"]])
max_rate_movies = average_rating[average_rating["Rating"] == max_rating]
pd.merge(max_rate_movies, movies)

,MovieID,Rating,Title,Genres,Year
0,787,5.0,"Gate of Heavenly Peace, The (1995)",Documentary,1995
1,989,5.0,Schlafes Bruder (Brother of Sleep) (1995),Drama,1995
2,1830,5.0,Follow the Bitch (1998),Comedy,1998
3,3172,5.0,Ulysses (Ulisse) (1954),Adventure,1954
4,3233,5.0,Smashing Time (1967),Comedy,1967
5,3280,5.0,"Baby, The (1973)",Horror,1973
6,3382,5.0,Song of Freedom (1936),Drama,1936
7,3607,5.0,One Little Indian (1973),Comedy|Drama|Western,1973
8,3656,5.0,Lured (1947),Crime,1947
9,3881,5.0,Bittersweet Motel (2000),Documentary,2000


### F. wykonaj poprzedni punkt, odrzucając wcześniej filmy które nie uzyskały wystarczająco dużo głosów (np 100)

In [145]:
result = pd.pivot_table(ratings[["MovieID", "Rating"]], index=["MovieID"], aggfunc=["mean", "size"]).reset_index()
result.head()
max_rating = float(result[result[("size", 0)] > 100].loc[0, [("mean", "Rating")]])
max_rate_movies = result[result[("mean", "Rating")] == max_rating]
max_rate_movies.head()
pd.merge(max_rate_movies, movies, left_on="MovieID", right_on="MovieID")

/home/adam/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,MovieID,"(MovieID, )","(mean, Rating)","(size, 0)",Title,Genres,Year
0,1,1,4.146846,2077,Toy Story (1995),Animation|Children's|Comedy,1995


### G. jaki jest najlepszy film według kobiet i według mężczyzn

In [168]:
data = pd.merge(pd.merge(ratings, movies), users)
pivot = pd.pivot_table(data, index=("Gender", "Title"), values=["Rating"], aggfunc="mean").reset_index().sort_values(by="Rating", ascending=False)
pivot.drop_duplicates(subset="Gender")

,Gender,Title,Rating
683,F,Coldblooded (1995),5.0
3820,M,"Bells, The (1926)",5.0


### H. jaki jest średni rok oglądanego filmu w poszczególnych grupach wiekowych

In [170]:
data = pd.merge(pd.merge(ratings, movies), users)

In [171]:
pd.pivot_table(data[["Age", "Year"]], index="Age", aggfunc="mean")

,Year
Age,
1,1988.981699
18,1989.701982
25,1987.972972
35,1984.965478
45,1983.500520
50,1982.483211
56,1981.549097


### I. jakie trzy gatunki filmowe są najczęściej oglądane przez kobiety i mężczyzn

In [176]:
genres = movies["Genres"].str.split("|", expand=True)
data = pd.concat([movies, genres], axis=1)
data.head()

,MovieID,Title,Genres,Year,0,1,2,3,4,5
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,Animation,Children's,Comedy,None,None,None
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,Adventure,Children's,Fantasy,None,None,None
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,Comedy,Romance,None,None,None,None
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,Comedy,Drama,None,None,None,None
4,5,Father of the Bride Part II (1995),Comedy,1995,Comedy,None,None,None,None,None


In [178]:
data = pd.melt(data, id_vars=["MovieID", "Title", "Genres", "Year"], value_vars=[0,1,2,3,4,5])

In [179]:
data_merge = pd.merge(pd.merge(data, ratings), users)

In [194]:
data_merge =  data_merge.groupby(["Gender", "value"], as_index=False).size().reset_index()

In [192]:
data_merge[data_merge["Gender"] == "F"].sort_values(by=0, ascending=False)[:3]

,Gender,value,0
7,F,Drama,98153
4,F,Comedy,96271
13,F,Romance,50297


In [193]:
data_merge[data_merge["Gender"] == "M"].sort_values(by=0, ascending=False)[:3]

,Gender,value,0
22,M,Comedy,260309
25,M,Drama,256376
18,M,Action,211807
